<a href="https://colab.research.google.com/github/shardul28/Document-similarity-K-shingles-minhashing-LSH-python/blob/master/MODEL_wawaweewa_XGBBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [3]:
#!pip install mit-news-classify


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
import pickle
#from mitnewsclassify import tfidf as mitx
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict,Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cd /content/drive/MyDrive/project_data

/content/drive/MyDrive/project_data


# MODEL


In [8]:
infile = open('/content/drive/MyDrive/project_data/clean_data.pkl','rb')
df = pickle.load(infile)

In [9]:
df.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0,army officer personnel killed action encounter...,"[army, officer, personnel, killed, action, enc...","[army, officer, personnel, killed, action, enc...",an army officer and four other personnel were ...
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0,aryan khan bail arrest week ago drugs cruise b...,"[aryan, khan, bail, arrest, week, ago, drug, c...","[aryan, khan, bail, arrest, week, ago, drug, c...",aryan khan did not get bail today for the thir...
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0,resigning national conference devender rana he...,"[resigning, national, conference, devender, ra...","[resigning, national, conference, devender, ra...",a day after resigning from the national confer...
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0,china artificial intelligence battle united st...,"[china, artificial, intelligence, battle, unit...","[china, artificial, intelligence, battle, unit...",china has won the artificial intelligence batt...
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0,various world leaders agencies posted wishes m...,"[various, world, leader, agency, posted, wish,...","[various, world, leader, agency, posted, wish,...",various world leaders and un agencies posted w...


In [10]:
df['clean_content']=""
for i in tqdm(range(len(df['clean_text_lemmatized']))):
    x=df['clean_text_lemmatized'][i]
    review=" ".join(x)
    df['clean_content'][i] = review

  0%|          | 0/2150 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 2150/2150 [00:01<00:00, 1709.16it/s]


In [11]:
df.head()

,text,url,title,content,bias,retweet_count,favorite_count,source_name,source,clean_content,text_lemmatized,clean_text_lemmatized,clean_content_with_Stopwords
0,"Army Officer, 4 Others Killed In Encounter In ...",https://www.ndtv.com/india-news/army-officer-4...,"Army Officer, 4 Personnel Killed In Encounter ...",an army officer and four other personnel were ...,3,9.0,63.0,ndtv,0,army officer personnel killed action encounter...,"[army, officer, personnel, killed, action, enc...","[army, officer, personnel, killed, action, enc...",an army officer and four other personnel were ...
1,"#AryanKhan Case: No Bail For Him Today, Next H...",https://ndtv.com/india-news/mumbai-drug-bust-c...,"Aryan Khan Case: No Bail For Him Today, Next H...",aryan khan did not get bail today for the thir...,3,5.0,44.0,ndtv,0,aryan khan bail arrest week ago drug cruise ba...,"[aryan, khan, bail, arrest, week, ago, drug, c...","[aryan, khan, bail, arrest, week, ago, drug, c...",aryan khan did not get bail today for the thir...
2,RT @ndtvfeed: Ex-National Conference Leaders D...,https://www.ndtv.com/india-news/after-resignin...,"Ex-National Conference Leaders Devender Rana, ...",a day after resigning from the national confer...,1,2.0,0.0,ndtv,0,resigning national conference devender rana he...,"[resigning, national, conference, devender, ra...","[resigning, national, conference, devender, ra...",a day after resigning from the national confer...
3,"RT @ndtvfeed: US Has Lost AI Battle To China, ...",https://www.ndtv.com/world-news/us-has-lost-ar...,"US Has Lost AI Battle To China, Pentagon's Ex-...",china has won the artificial intelligence batt...,0,5.0,0.0,ndtv,0,china artificial intelligence battle united he...,"[china, artificial, intelligence, battle, unit...","[china, artificial, intelligence, battle, unit...",china has won the artificial intelligence batt...
4,RT @ndtvfeed: A Look At Inspiring Messages On ...,https://www.ndtv.com/india-news/on-internation...,International Day Of The Girl Child 2021: Lead...,various world leaders and un agencies posted w...,3,3.0,0.0,ndtv,0,various world leader agency posted wish messag...,"[various, world, leader, agency, posted, wish,...","[various, world, leader, agency, posted, wish,...",various world leaders and un agencies posted w...


In [12]:
# from keras.preprocessing.sequence import pad_sequences
# max_features = 30000
# maxlen = 25
# X = df['clean_text_lemmatized']
# X = pad_sequences(X, maxlen = maxlen)
# y=df['bias']

In [13]:
for x in df['clean_text_lemmatized'][2]:
   if(x=='car'):
      print(x)

In [14]:
for i in range(len(df)):
  for x in df['clean_text_lemmatized'][i]:
    if x=='car':
      print(i)
      


In [15]:
df['content'][0]

"an army officer and four other personnel were killed in action during an encounter between security forces and terrorists in jammu and kashmir's poonch district on monday, officials said. an anti-insurgency operation was launched in a village close to dera ki gali in surankote in the early hours following intelligence inputs about the presence of heavily-armed terrorists there, they said. the hiding terrorists fired on the search parties resulting in fierce gunfight and critical injuries to a junior commissioned officer or jco and four other soldiers. they died from their injuries. the encounter with the terrorists was going on and further details are awaited, a defence spokesperson said. officials said there were reports about the presence of a group of heavily-armed terrorists in the chamrer forest, after they managed to sneak from across the line of control (loc). reinforcements have been rushed to the area to plug all escape routes of the terrorists, they said. (with inputs from p

In [16]:
#df['clean_text_lemmatized'][0]

In [17]:
#//

In [18]:
for i in range(len(df)):
  for x in df['clean_text_lemmatized'][i]:
    if str(x)=='car':
      print('here',i)


In [19]:
center_with_bias_words = df.loc[df['content'].str.contains('shiv sena|shivsena|aap|bjp|congress|modi|gandhi', case=False)]
center_without_bias_words = df.loc[~df['content'].str.contains('sena|shivsena|aap|bjp|congress|modi|gandhi|minister|aam|government|aadmi', case=False)]

center_without_bias_words = center_without_bias_words.loc[center_without_bias_words['bias']==3]

center_with_bias_words = center_with_bias_words.loc[center_with_bias_words['bias']==3]
right = df.loc[df['bias']==1]
left = df.loc[df['bias']==0]
center = df.loc[df['bias']==3]
#center = df.loc[df['bias']==3]
#df2=pd.concat([left,right,center_with_bias_words])


In [20]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



right 184
left 263
center_without_bias_words 811
center_with_bias_words 443


In [21]:
# for xt in all_words.most_common(5000):
#   print(xt)###

In [22]:
#left.loc[left['clean_content'].str.contains('power')]#, case=False)]


In [23]:
#right.loc[right['clean_content'].str.contains('power')]#, case=False)]

In [24]:
left=left.head(190)

In [25]:
center_with_bias_words=center_with_bias_words.head(200)

In [26]:
center_without_bias_words=center_without_bias_words.head(200)


In [27]:
center=center.head(300)

In [28]:
print('right',len(right))
print('left',len(left))
print('center_without_bias_words',len(center_without_bias_words))
print('center_with_bias_words',len(center_with_bias_words))



right 184
left 190
center_without_bias_words 200
center_with_bias_words 200


In [29]:
right['clean_content'][2]

'resigning national conference devender rana head jammu division party leader surjit singh slathia joined bjp rana slathia joined bjp headquarters delhi presence union minister dharmendra pradhan hardeep singh puri jitendra singh rana mla younger brother union minister jitendra singh served political advisor jammu kashmir chief minister national conference nc leader omar abdullah bjp general secretary charge jammu kashmir tarun chugh jammu kashmir unit chief ravinder raina welcomed party rana advocating jammu declaration joint declaration political social business organisation primarily demanding restoration statehood jammu region jammu kashmir centre revoked erstwhile jammu kashmir special status bifurcated union territory jammu kashmir ladakh'

# DF CONCAT

In [30]:
#df2=pd.concat([left,right,center])
df2=pd.concat([left,right,center_without_bias_words])


In [31]:
#REINDEXING
indx=[]
for i in range(len(df2)):
  indx.append(i)
df2.index=indx

In [32]:
# ## Divide the dataset into Train and Test
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2)

In [33]:
######NaiiveBayes without Feature Selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score,classification_report,accuracy_score,confusion_matrix
from sklearn.naive_bayes import GaussianNB
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [34]:
X = df2['clean_content']
y=df2['bias']


In [35]:
from nltk.tokenize import word_tokenize
all_words = []
for message in df2['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)

In [36]:
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 11170
Most common words: [('minister', 1015), ('farmer', 949), ('congress', 714), ('bjp', 663), ('party', 643), ('government', 607), ('chief', 599), ('police', 569), ('leader', 558), ('lakhimpur', 514), ('modi', 513), ('gandhi', 494), ('union', 449), ('mishra', 430), ('kheri', 420)]


# COMMON WORDS TESTING


In [37]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in right['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(2500)))

Number of words: 5953
Most common words: [('minister', 422), ('bjp', 356), ('farmer', 321), ('modi', 314), ('party', 290), ('congress', 264), ('union', 223), ('government', 222), ('leader', 216), ('chief', 208), ('singh', 167), ('lakhimpur', 160), ('mishra', 151), ('son', 138), ('gandhi', 135), ('pradesh', 130), ('pm', 125), ('kheri', 117), ('police', 106), ('national', 101), ('power', 100), ('uttar', 98), ('home', 94), ('election', 89), ('assembly', 86), ('ashish', 83), ('delhi', 82), ('ajay', 80), ('shah', 69), ('cm', 68), ('kumar', 67), ('worker', 66), ('kashmir', 65), ('violence', 65), ('meeting', 62), ('claimed', 60), ('protest', 60), ('ncb', 59), ('issue', 58), ('come', 57), ('law', 57), ('yadav', 57), ('indian', 56), ('killed', 56), ('development', 55), ('death', 54), ('present', 54), ('khan', 54), ('punjab', 54), ('political', 53), ('ministry', 53), ('drug', 53), ('rjd', 53), ('jammu', 52), ('killing', 51), ('opposition', 50), ('president', 50), ('place', 50), ('held', 50), ('p

In [38]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in left['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(2500)))

Number of words: 5088
Most common words: [('farmer', 626), ('minister', 593), ('congress', 450), ('government', 385), ('gandhi', 359), ('lakhimpur', 354), ('leader', 329), ('chief', 322), ('party', 314), ('bjp', 307), ('kheri', 303), ('mishra', 277), ('police', 237), ('son', 220), ('pradesh', 218), ('union', 215), ('uttar', 207), ('modi', 199), ('priyanka', 195), ('violence', 180), ('power', 165), ('singh', 159), ('ajay', 151), ('vadra', 147), ('ashish', 145), ('protest', 143), ('delhi', 134), ('killed', 131), ('election', 120), ('told', 106), ('order', 98), ('alleged', 96), ('home', 94), ('law', 88), ('meet', 87), ('centre', 85), ('rahul', 85), ('place', 83), ('punjab', 83), ('arrest', 83), ('lucknow', 81), ('died', 80), ('member', 78), ('assembly', 77), ('worker', 77), ('deputy', 76), ('opposition', 76), ('court', 75), ('fir', 74), ('political', 71), ('kumar', 70), ('accused', 70), ('detained', 70), ('general', 69), ('sidhu', 65), ('saying', 64), ('want', 64), ('justice', 64), ('twee

In [39]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in center_without_bias_words['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(2500)))

Number of words: 6667
Most common words: [('police', 226), ('court', 195), ('accused', 143), ('drug', 141), ('khan', 139), ('woman', 134), ('official', 127), ('aryan', 97), ('agency', 96), ('bail', 87), ('mumbai', 82), ('high', 74), ('ncb', 69), ('chief', 69), ('cruise', 68), ('told', 68), ('investigation', 66), ('according', 64), ('china', 63), ('indian', 59), ('anti', 57), ('old', 57), ('health', 56), ('custody', 54), ('week', 53), ('chinese', 53), ('delhi', 53), ('medium', 52), ('control', 51), ('bureau', 51), ('world', 51), ('justice', 50), ('national', 49), ('place', 48), ('member', 48), ('number', 48), ('russia', 48), ('statement', 47), ('month', 47), ('company', 46), ('act', 45), ('home', 44), ('social', 44), ('air', 44), ('prize', 44), ('lawyer', 43), ('sent', 43), ('narcotic', 42), ('reported', 42), ('issue', 42), ('gram', 42), ('facebook', 42), ('group', 41), ('child', 41), ('life', 41), ('party', 39), ('took', 39), ('later', 39), ('house', 39), ('ship', 39), ('jail', 38), ('

In [40]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in center_with_bias_words['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(2500)))

Number of words: 7923
Most common words: [('minister', 394), ('government', 338), ('modi', 258), ('farmer', 254), ('court', 242), ('congress', 205), ('bjp', 199), ('party', 197), ('chief', 189), ('power', 176), ('leader', 172), ('son', 166), ('police', 150), ('mishra', 148), ('told', 128), ('company', 125), ('president', 121), ('union', 120), ('lakhimpur', 111), ('election', 108), ('world', 105), ('official', 104), ('air', 103), ('group', 102), ('law', 100), ('khan', 99), ('facebook', 99), ('medium', 96), ('issue', 95), ('gandhi', 95), ('month', 94), ('pradesh', 94), ('violence', 91), ('tata', 90), ('ncb', 89), ('high', 88), ('singh', 88), ('kheri', 88), ('drug', 87), ('action', 86), ('home', 84), ('uttar', 82), ('according', 81), ('worker', 80), ('pakistan', 79), ('right', 77), ('come', 77), ('week', 76), ('political', 76), ('justice', 76), ('national', 75), ('million', 75), ('death', 75), ('delhi', 74), ('member', 74), ('indian', 74), ('protest', 73), ('agency', 71), ('statement', 71

In [41]:
###TESTING
from nltk.tokenize import word_tokenize
all_words = []
for message in center['clean_content']:
      words = word_tokenize(message)
      for w in words:
          all_words.append(w)
all_words = nltk.FreqDist(all_words)
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(2500)))

Number of words: 8994
Most common words: [('police', 301), ('court', 265), ('government', 251), ('official', 216), ('air', 194), ('accused', 170), ('minister', 159), ('woman', 157), ('tata', 151), ('world', 147), ('chief', 147), ('power', 146), ('khan', 129), ('drug', 129), ('agency', 125), ('china', 125), ('group', 124), ('high', 115), ('health', 115), ('mumbai', 114), ('told', 106), ('indian', 105), ('statement', 104), ('according', 100), ('bail', 97), ('issue', 97), ('son', 96), ('airline', 96), ('week', 94), ('medium', 94), ('aryan', 89), ('airport', 88), ('president', 86), ('month', 86), ('modi', 86), ('leader', 85), ('international', 85), ('national', 83), ('singh', 83), ('member', 82), ('afghanistan', 82), ('taliban', 81), ('justice', 81), ('control', 80), ('ncb', 80), ('right', 78), ('law', 78), ('say', 78), ('delhi', 78), ('work', 75), ('force', 74), ('journalist', 74), ('company', 73), ('killed', 71), ('security', 71), ('old', 71), ('united', 71), ('attack', 71), ('girl', 70)

In [42]:
#///

# TFIDF

In [43]:
## TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=20000,ngram_range=(1,3))##35##69

In [44]:
X=tfidf_v.fit_transform(df2['clean_content']).toarray()


In [45]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [46]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 ,random_state=42,stratify=y)

In [47]:
count_df = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())
count_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aadmi,aadmi party,aadmi party aap,aadmi party claimed,aam,aam aadmi,aam aadmi party,aap,aap government,...,zp panchayat samitis,zuckerberg,zuckerberg cover,zuckerberg cover latest,zuckerberg hit,zuckerberg hit whistleblower,zuckerberg wrote,zuckerberg wrote note,zurich,zyklon
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
y_train.value_counts()

3    134
0    127
1    123
Name: bias, dtype: int64

In [49]:
y_test.value_counts()


3    66
0    63
1    61
Name: bias, dtype: int64

In [50]:
y_test.value_counts()

3    66
0    63
1    61
Name: bias, dtype: int64

In [51]:
#///

In [118]:
from sklearn import model_selection, naive_bayes, svm
# fit the training dataset on the NB classifier
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=30,learning_rate=1)#,random_state=5)

xgb.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_xgb = xgb.predict(X_test)
# Use accuracy_score function to get the accuracy
print("xgb Accuracy Score -> ",accuracy_score(predictions_xgb, y_test)*100)
print(classification_report(y_test,predictions_xgb))

xgb Accuracy Score ->  82.10526315789474
              precision    recall  f1-score   support

           0       0.74      0.73      0.74        63
           1       0.75      0.72      0.73        61
           3       0.96      1.00      0.98        66

    accuracy                           0.82       190
   macro avg       0.81      0.82      0.82       190
weighted avg       0.82      0.82      0.82       190



In [115]:
from sklearn import model_selection, naive_bayes, svm
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB(alpha=1,fit_prior=False)
Naive.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)
print(classification_report(y_test,predictions_NB))

Naive Bayes Accuracy Score ->  78.42105263157895
              precision    recall  f1-score   support

           0       0.67      0.90      0.77        63
           1       0.86      0.49      0.62        61
           3       0.89      0.94      0.91        66

    accuracy                           0.78       190
   macro avg       0.80      0.78      0.77       190
weighted avg       0.81      0.78      0.77       190



In [ ]:
rf_classifier=RandomForestClassifier().fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


In [54]:
rf_classifier=RandomForestClassifier( max_features='auto', min_samples_leaf=4, min_samples_split=5, n_estimators=1000).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


[[48  7  8]
 [21 33  7]
 [ 0  0 66]]
0.7736842105263158
              precision    recall  f1-score   support

           0       0.70      0.76      0.73        63
           1       0.82      0.54      0.65        61
           3       0.81      1.00      0.90        66

    accuracy                           0.77       190
   macro avg       0.78      0.77      0.76       190
weighted avg       0.78      0.77      0.76       190



In [55]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=300).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)
y.value_counts()
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))


[[52  4  7]
 [19 37  5]
 [ 0  0 66]]
0.8157894736842105
              precision    recall  f1-score   support

           0       0.73      0.83      0.78        63
           1       0.90      0.61      0.73        61
           3       0.85      1.00      0.92        66

    accuracy                           0.82       190
   macro avg       0.83      0.81      0.81       190
weighted avg       0.83      0.82      0.81       190



In [56]:
# //

# RF  Manual Hyperparameter Tuning

In [57]:
# ### Manual Hyperparameter Tuning
# # {'criterion': 'entropy',
# #  'max_depth': 100,
# #  'max_features': 'sqrt',
# #  'min_samples_leaf': 1,
# #  'min_samples_split': 2,
# #  'n_estimators': 600}
# model=RandomForestClassifier(n_estimators=900,criterion='entropy',
#                              max_features='sqrt',min_samples_leaf=1,min_samples_split=2,random_state=100).fit(X_train,y_train)
# predictions=model.predict(X_test)
# print(confusion_matrix(y_test,predictions))
# print(accuracy_score(y_test,predictions))
# print(classification_report(y_test,predictions))

#RF WITH RANDOMSEARCH CV

RF WITH RANDOMSEARCH CV

In [58]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 500,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [59]:
//

SyntaxError: ignored

In [ ]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=20,cv=3,verbose=2,
                               random_state=50,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

In [ ]:
rf_randomcv.best_params_

In [ ]:
best_random_grid=rf_randomcv.best_estimator_
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))

In [ ]:
//

#RF WITH GridSearchCV


RF WITH GridSearchCV


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)


In [ ]:
//

# RF WITH TPOT OPTIMIZATION

In [ ]:
pip install tpot


In [ ]:
from tpot import TPOTClassifier

tpot_classifier = TPOTClassifier(generations= 3, population_size= 30, offspring_size= 10,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier':param}, 
                                 cv = 4, scoring = ['accuracy','recall','precision'])
tpot_classifier.fit(X_train,y_train)

In [ ]:
### Manual Hyperparameter Tuning
# {'criterion': 'entropy',
#  'max_depth': 100,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 600}
#model=RandomForestClassifier( criterion='gini', max_depth=340, max_features='auto', min_samples_leaf=2, min_samples_split=5, n_estimators=2000).fit(X_train,y_train)
model=RandomForestClassifier(criterion='entropy', max_depth=780, max_features='auto', min_samples_leaf=4, min_samples_split=5, n_estimators=1000).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
pip install optuna


In [ ]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

In [ ]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=text_clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: \n {}".format(classification_report(y_test,y_pred)))

# EXTRA

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)
clfs = {
    'SVC' : svc,
    'KN' : knc, 
    'NB': mnb, 
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}
def train_classifier(clf,X_train,y_train,X_test,y_test):
    print(clf)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred,pos_label='positive',average=None)
    recall =recall_score(y_test, y_pred,pos_label='positive',average=None)
    return accuracy,precision,recall
    
train_classifier(svc,X_train,y_train,X_test,y_test)


SVC(gamma=1.0, kernel='sigmoid')


(0.8,
 array([0.74242424, 0.79166667, 0.85526316]),
 array([0.77777778, 0.62295082, 0.98484848]))

In [73]:
import warnings
warnings.filterwarnings("ignore")

In [78]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():
    current_accuracy,current_precision,current_recall = train_classifier(clf, X_train,y_train,X_test,y_test)
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)
    print("Recall - ",current_recall)

    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

SVC(gamma=1.0, kernel='sigmoid')
For  SVC
Accuracy -  0.8
Precision -  [0.74242424 0.79166667 0.85526316]
Recall -  [0.77777778 0.62295082 0.98484848]
KNeighborsClassifier()
For  KN
Accuracy -  0.7263157894736842
Precision -  [0.63636364 0.68627451 0.87096774]
Recall -  [0.77777778 0.57377049 0.81818182]
MultinomialNB()
For  NB
Accuracy -  0.7789473684210526
Precision -  [0.6627907  0.9        0.86486486]
Recall -  [0.9047619  0.44262295 0.96969697]
DecisionTreeClassifier(max_depth=5)
For  DT
Accuracy -  0.7421052631578947
Precision -  [0.62711864 0.62295082 0.94285714]
Recall -  [0.58730159 0.62295082 1.        ]
LogisticRegression(penalty='l1', solver='liblinear')
For  LR
Accuracy -  0.6947368421052632
Precision -  [0.60273973 0.6875     0.77647059]
Recall -  [0.6984127  0.36065574 1.        ]
RandomForestClassifier(n_estimators=50, random_state=2)
For  RF
Accuracy -  0.8157894736842105
Precision -  [0.73239437 0.925      0.83544304]
Recall -  [0.82539683 0.60655738 1.        ]
AdaBo